### H3 - Existe algum tipo de relação entre produtos que gera uma compra "combinada"?

#### Importando pacotes

In [23]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

#### Lendo arquivos

In [7]:
arquivo = "../_data/clean/e_commerce_pedidos.csv"

In [92]:
pedidos_df = pd.read_csv(arquivo)

In [93]:
pedidos_df.head()

,Unnamed: 0,id,data,data_pagamento,data_cancelamento,numero_pedido,status,pagamento,produto,sku,...,entrega_cidade,entrega_estado,entrega_cep,utm_source,utm_campaign,utm_medium,utm_content,utm_term,customizacao,id_cliente
0,1,117569438,04/08/2024 18:26:40,04/08/2024 18:27,NaN,1042552530523300,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Brasilia,Distrito Federal,72455490,NaN,NaN,NaN,NaN,NaN,NaN,31d69aed10
1,2,117385907,01/08/2024 19:08:02,01/08/2024 19:11,NaN,1042552893870166,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
2,3,117385794,01/08/2024 19:05:53,NaN,01/08/2024 19:12,1042552190094688,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
3,4,117382873,01/08/2024 18:12:14,NaN,01/08/2024 19:13,1042552273445602,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
4,5,117382771,01/08/2024 18:09:41,NaN,01/08/2024 19:13,1042552782939597,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5


#### Selecionando colunas relevantes
Esta tabela está a nível de sku. Assim temos para cada linha o sku que está incluso no pedido.

In [94]:
pedidos = pedidos_df[['numero_pedido', 'sku']]

In [95]:
pedidos.head()

,numero_pedido,sku
0,1042552530523300,DFREFSP8P
1,1042552893870166,DFREFSP8P
2,1042552190094688,DFREFSP8P
3,1042552273445602,DFREFSP8P
4,1042552782939597,DFREFSP8P


### Analisando produtos comprados em conjunto

#### Lista de produtos por pedido

In [96]:
df_produtos_por_pedido = pedidos.groupby('numero_pedido')['sku'].apply(list).reset_index()

In [97]:
df_produtos_por_pedido.head()

,numero_pedido,sku
0,1042552147040598,"[HL3J67375, 2QALTPNGA, 84WFBE5GN]"
1,1042552155461445,[JRM3YWN95]
2,1042552166282916,"[84WFBE5GN, 6XY5VVVRE]"
3,1042552190094688,[DFREFSP8P]
4,1042552221071163,"[JRM3YWN95, TMXCUUW38, E7SLSQLCC, HRPKRPNYD, T..."


#### Contabilizando produtos vendidos em conjunto

In [100]:
# Realiza o auto join para encontrar os produtos vendidos juntos no mesmo pedido
df_joined = pedidos.merge(pedidos, on="numero_pedido")

# Filtra as linhas onde os produtos são diferentes
df_joined = df_joined[df_joined['sku_x'] != df_joined['sku_y']]

# Agrupa e conta as combinações de produtos
df_resultado = df_joined.groupby(['sku_x', 'sku_y'])\
                        .agg(contagem=('numero_pedido', 'size'))\
                        .reset_index()

# Renomeia as colunas
df_resultado = df_resultado.rename(columns={'sku_x': 'sku', 'sku_y': 'sku_casado'})

# Ordena o resultado
df_resultado = df_resultado.sort_values(by=['contagem'], ascending=False)

In [101]:
df_joined.head()

,numero_pedido,sku_x,sku_y
17,1042552760701491,PPW5U8MVC,LSRE3YWF5
18,1042552760701491,LSRE3YWF5,PPW5U8MVC
23,1042552772676481,P7HXNQNUN,T5LJ2R3L6
24,1042552772676481,P7HXNQNUN,84WFBE5GN
25,1042552772676481,T5LJ2R3L6,P7HXNQNUN


In [102]:
df_resultado.head()

,sku,sku_casado,contagem
48,D56A2X2M7,F8ZW4347R,3
13,3LP9NDQHZ,D56A2X2M7,3
46,D56A2X2M7,3LP9NDQHZ,3
47,D56A2X2M7,74F6CQWEW,3
122,P7HXNQNUN,84WFBE5GN,3


### Similaridade de Produtos

In [103]:
# Tabela Pedido, Produto e Quantidade
similaridade_produtos = pedidos_df[['numero_pedido', 'sku', 'quantidade']]

In [104]:
similaridade_produtos = similaridade_produtos.groupby(['numero_pedido', 'sku'])\
                        .agg(quantidade=('quantidade', 'size'))\
                        .reset_index()\
                        .sort_values(by=['quantidade'], ascending=False)

In [105]:
similaridade_produtos.head()

,numero_pedido,sku,quantidade
0,1042552147040598,2QALTPNGA,1
73,1042552616727187,F8KH5F8MN,1
93,1042552711601682,P7HXNQNUN,1
94,1042552727006222,4P8E3PJHB,1
95,1042552730060143,3LP9NDQHZ,1


In [106]:
# Criando uma matriz de utilidade onde:
# - Linhas representam pedidos
# - Colunas representam produtos
# - Valores representam a quantidade comprada
matriz_utilidade = similaridade_produtos.pivot_table(index='sku',
                                       values='quantidade',
                                       columns='numero_pedido',
                                       fill_value=0)

In [107]:
matriz_utilidade.head()

numero_pedido,1042552147040598,1042552155461445,1042552166282916,1042552190094688,1042552221071163,1042552243758772,1042552260877262,1042552266423422,1042552273445602,1042552306098971,...,1042552912381732,1042552916150952,1042552919766370,1042552931198287,1042552940819893,1042552948986644,1042552959879245,1042552979257588,1042552979344315,1042552982761575
sku,,,,,,,,,,,,,,,,,,,,,
2QALTPNGA,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2XZUH896Q,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3LP9NDQHZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4P8E3PJHB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6XY5VVVRE,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
# Calculando a similaridade cosseno entre os produtos com base em seus padrões de compra.
matriz_similaridade = cosine_similarity(matriz_utilidade)

In [109]:
print(matriz_similaridade)

[[1.         0.         0.21821789 0.         0.         0.
  0.18257419 0.         0.         0.33333333 0.         0.
  0.         0.         0.         0.         0.         0.23570226
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.23570226
  0.25819889 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.18257419 0.         0.
  0.4330127  0.         0.23570226 0.         0.         0.21821789]
 [0.21821789 0.         1.         0.         0.         0.32732684
  0.11952286 0.         0.56694671 0.43643578 0.         0.18898224
  0.         0.         0.56694671 0.         0.         0.
  0.         0.         0.         0.16903085 0.         0.
  0.13363062 0.         0.         0.         0.21821789 0.        ]
 [0.         0.         0.         1.    

In [110]:
# Convertendo a matriz de similaridade em um DataFrame
df_similaridade = pd.DataFrame(matriz_similaridade, index=matriz_utilidade.index, columns=matriz_utilidade.index)

In [111]:
df_similaridade.head()

sku,2QALTPNGA,2XZUH896Q,3LP9NDQHZ,4P8E3PJHB,6XY5VVVRE,74F6CQWEW,84WFBE5GN,9M294TUTK,D56A2X2M7,DAGRBZZ8K,...,JRM3YWN95,JWDBL2NB2,LSRE3YWF5,M7ZQ5L9JL,P7HXNQNUN,PPW5U8MVC,T5LJ2R3L6,T76T2L3YK,THDCJT4YR,TMXCUUW38
sku,,,,,,,,,,,,,,,,,,,,,
2QALTPNGA,1.000000,0.0,0.218218,0.0,0.0,0.000000,0.182574,0.0,0.000000,0.333333,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2XZUH896Q,0.000000,1.0,0.000000,0.0,0.0,0.235702,0.258199,0.0,0.000000,0.000000,...,0.0,0.182574,0.0,0.0,0.433013,0.0,0.235702,0.0,0.000000,0.218218
3LP9NDQHZ,0.218218,0.0,1.000000,0.0,0.0,0.327327,0.119523,0.0,0.566947,0.436436,...,0.0,0.169031,0.0,0.0,0.133631,0.0,0.000000,0.0,0.218218,0.000000
4P8E3PJHB,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
6XY5VVVRE,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.365148,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000


In [112]:
# Adicionando uma coluna 'sku' ao DataFrame de similaridade para fácil referência.
df_similaridade['sku'] = df_similaridade.index

In [113]:
# Reordenando as colunas para que a coluna 'produto' seja a primeira.
cols = ['sku'] + [col for col in df_similaridade if col != 'sku']
df_similaridade = df_similaridade[cols]

In [114]:
df_similaridade.head()

sku,sku,2QALTPNGA,2XZUH896Q,3LP9NDQHZ,4P8E3PJHB,6XY5VVVRE,74F6CQWEW,84WFBE5GN,9M294TUTK,D56A2X2M7,...,JRM3YWN95,JWDBL2NB2,LSRE3YWF5,M7ZQ5L9JL,P7HXNQNUN,PPW5U8MVC,T5LJ2R3L6,T76T2L3YK,THDCJT4YR,TMXCUUW38
sku,,,,,,,,,,,,,,,,,,,,,
2QALTPNGA,2QALTPNGA,1.000000,0.0,0.218218,0.0,0.0,0.000000,0.182574,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2XZUH896Q,2XZUH896Q,0.000000,1.0,0.000000,0.0,0.0,0.235702,0.258199,0.0,0.000000,...,0.0,0.182574,0.0,0.0,0.433013,0.0,0.235702,0.0,0.000000,0.218218
3LP9NDQHZ,3LP9NDQHZ,0.218218,0.0,1.000000,0.0,0.0,0.327327,0.119523,0.0,0.566947,...,0.0,0.169031,0.0,0.0,0.133631,0.0,0.000000,0.0,0.218218,0.000000
4P8E3PJHB,4P8E3PJHB,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
6XY5VVVRE,6XY5VVVRE,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.365148,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000


In [115]:
# Despivotando a tabela
df_unpivot = df_similaridade.melt(id_vars=['sku'], var_name='sku_casado', value_name='similaridade')\
                             .sort_values(by=['similaridade'], ascending=False)

#Filtrando produtos diferentes
df_unpivot = df_unpivot[df_unpivot['sku'] != df_unpivot['sku_casado']]

In [116]:
display(df_unpivot)

,sku,sku_casado,similaridade
254,F8ZW4347R,D56A2X2M7,0.75000
428,D56A2X2M7,F8ZW4347R,0.75000
568,THDCJT4YR,HRPKRPNYD,0.57735
851,E7SLSQLCC,THDCJT4YR,0.57735
858,HRPKRPNYD,THDCJT4YR,0.57735
...,...,...,...
349,JMPBRU537,E7SLSQLCC,0.00000
352,LSRE3YWF5,E7SLSQLCC,0.00000
353,M7ZQ5L9JL,E7SLSQLCC,0.00000
354,P7HXNQNUN,E7SLSQLCC,0.00000


### Resultados

Produtos comprados em conjunto:

In [117]:
#Produto e descrição
produtos = pedidos_df[['sku', 'produto']].drop_duplicates(subset=['sku', 'produto'])


produtos.head()

,sku,produto
0,DFREFSP8P,BRACELETE AJUSTÁVEL O COM COROA
8,M7ZQ5L9JL,BRACELETE AJUSTÁVEL O SIGNATURE
9,JRM3YWN95,BRINCO ARGOLA SOL E LUA
11,TMXCUUW38,BRINCO PINGENTE SIGNATURE
12,PPW5U8MVC,BRINCO CORAÇÕES


In [120]:
df_resultado_prod_conj = pd.merge(df_resultado, produtos, on='sku', how='left')

#df_resultado_prod_conj = df_resultado_prod_conj.merge(df_resultado_prod_conj, produtos, left_on='sku_casado', right_on='sku', how='left')

df_resultado_prod_conj.head()

,sku,sku_casado,contagem,produto
0,D56A2X2M7,F8ZW4347R,3,BRACELETE RÍGIDO 19
1,3LP9NDQHZ,D56A2X2M7,3,CHARM SUCESSO
2,D56A2X2M7,3LP9NDQHZ,3,BRACELETE RÍGIDO 19
3,D56A2X2M7,74F6CQWEW,3,BRACELETE RÍGIDO 19
4,P7HXNQNUN,84WFBE5GN,3,BRINCO QUADRADO CRAVEJADO


In [123]:
df_resultado_prod_conj2 = df_resultado_prod_conj.merge(df_resultado_prod_conj, produtos, right_on='sku', left_on='sku_casado', how='inner')

TypeError: merge() got multiple values for argument 'how'